In [175]:
from sklearn import preprocessing as pp
import numpy as np
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Lambda
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from numpy.random import multinomial as randm

# Current Counts = Current Counts + Note Length % 16 ?

def buildNotes():
    
    print("In buildNotes()")
    
    notes = []
    noteLengths = []
    
    for file in glob.glob("Music/*.mid"):
        
        try:
            midi = converter.parse(file)    
            
        except:
            print("MIDI file %s failed to parse" % file)
            continue
            
        print("Parsing %s" % file)
        notesToParse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notesToParse = s2.parts[0].recurse()
            print("Instrument Parts %d" % len(s2.parts))
            
        except: # file has notes in a flat structure
            notesToParse = midi.flat.notes
            
        # For each note in notesToParse (a stream of notes)
            
        for element in notesToParse:
            
            # Interesting parameters of notes that we may want to look at here.
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                noteLengths.append(str(element.quarterLength))
                
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                noteLengths.append(str(element.quarterLength))

    pickle.dump(notes, open('notes.p', 'wb'))

    return notes, noteLengths

def prepareSeq(notes, noteLengths):

    seqLength = 4 

    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))

    inputSize = len(notes) - seqLength
    
    noteIn = [[],[]]
    noteOut = [[],[]]

    # Mapping of seqLength notes to note i + seqLength
    
    # X[0] sequence_in : [seqLength]
    # X[1] length_in : [seqLength]
    
    # Y[0] sequence_out : 1
    # Y[1] length_out : 1
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        
        length_in = noteLengths[i:i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteIn[1].append([lengthToInt[char] for char in length_in])
        
        noteOut[0].append(noteToInt[sequence_out])
        noteOut[1].append(lengthToInt[length_out])

    # npatterns = len(noteIn) (inputSize)

    # reshape the input into a format compatible with LSTM layers
    # noteIn = np.reshape(noteIn, (inputSize, seqLength, 1))
    
    noteIn = np.array(noteIn)
    noteOut = np.array(noteOut)
    
    print(noteIn.shape)
    print(noteOut.shape)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
    
    catNote = np_utils.to_categorical(noteOut[0])
    catLength = np_utils.to_categorical(noteOut[1])
    
    # Concatenate the categorical arrays along axis 1 (columnwise)
    
    din = np.concatenate((noteIn[0], noteIn[1]))
    dout = np.concatenate((catNote, catLength), 1)
    
    din = np.reshape(din, (len(din), seqLength, 1))
    
    print(dout.shape)
    print(len(catNote[0]))
    print(len(catLength[0]))

    return (din, dout)


In [176]:
# print(np.arange(10).reshape(5,2).T.flatten())
notes, noteLengths = buildNotes()
noteIn, noteOut = prepareSeq(notes, noteLengths)
print(noteIn.shape)
print(noteOut.shape)

In buildNotes()
Parsing Music\Fugue1.mid
(2, 736, 4)
(2, 736)
(736, 58)
43
15
(1472, 4, 1)
(736, 58)


In [71]:
def createNet(noteIn, nvocab):

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    # Chris's code
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=False
    ))
    
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation(activation='relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    
    #model.add(Flatten())
    
    model.add(Dense(nvocab))
    model.add(Lambda(lambda x: x / 0.6))
    model.add(Activation(activation='softmax'))

    model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
    
    return model

def trainNet(_epochs=1):
    """ Train a Neural Network to generate music """
    notes, noteLengths = buildNotes()

    nvocab = len(set(notes))
    
    noteIn, noteOut = prepareSeq(notes, noteLengths, nvocab)
    
    model = createNet(noteIn, nvocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(noteIn, noteOut, batch_size=64, epochs=_epochs, verbose=1, callbacks=callbacks_list)
    
def prepareSeqPred(notes, pitchnames, nvocab):
    print("In Prepare Sequences Prediction")
    noteToInt = dict((note, number) for number, note in enumerate(pitchnames))
    seqLength = 4
    noteIn = []
    output = []
    for i in range(0, len(notes) - seqLength, 1):
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        noteIn.append([noteToInt[char] for char in sequence_in])
        output.append(noteToInt[sequence_out])

    npatterns = len(noteIn)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(noteIn, (npatterns, seqLength, 1))
    # normalize input
    normalized_input = normalized_input / float(nvocab)

    return (noteIn, normalized_input)

def genNotes(model, noteIn, pitchnames, nvocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(noteIn)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = noteIn[start]
    predOut = []
    
    print("In Generate Notes: noteIn[start] = %s" % pattern)

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(nvocab)

        
        ### Complete the line below
        prediction = model.predict(np.array(prediction_input))
        
        array = randm(1, prediction[0])
        index = np.argmax(array)
        
        result = int_to_note[index]
        predOut.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return predOut

def generate():
    print("In Generate")
    notes = pickle.load(open('notes.p', 'rb'))
    pitchnames = sorted(set(item for item in notes))
    nvocab = len(set(notes))

    noteIn, normalized_input = prepareSeqPred(notes, pitchnames, nvocab)
#    model = createNet(normalized_input, nvocab)
    
    ### Add a line to load the weights here
    model = load_model("weights2-improvement-01-4.0703-bigger.hdf5")
    
    predOut = genNotes(model, noteIn, pitchnames, nvocab)
    createMidi(predOut)
    
def createMidi(predOut):
    print("In Create Midi")
    offset = 0
    output_notes = []
    for pattern in predOut:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')

In [72]:
trainNet()

In buildNotes()
Parsing Music\Fugue1.mid
Parsing Music\Fugue10.mid
Parsing Music\Fugue11.mid
MIDI file Music\Fugue12.mid failed to parse
Parsing Music\Fugue13.mid
Parsing Music\Fugue14.mid
Parsing Music\Fugue15.mid
Parsing Music\Fugue16.mid
Parsing Music\Fugue17.mid
Parsing Music\Fugue18.mid
MIDI file Music\Fugue19.mid failed to parse
Parsing Music\Fugue2.mid
Parsing Music\Fugue20.mid
Parsing Music\Fugue21.mid
Parsing Music\Fugue22.mid
Parsing Music\Fugue23.mid
Parsing Music\Fugue24.mid
Parsing Music\Fugue3.mid
Parsing Music\Fugue4.mid
Parsing Music\Fugue5.mid
MIDI file Music\Fugue6.mid failed to parse
Parsing Music\Fugue7.mid
Parsing Music\Fugue8.mid
Parsing Music\Fugue9.mid
Instrument Parts 1
{'0.0': 0, '0.25': 1, '0.5': 2, '0.75': 3, '1.0': 4, '1.25': 5, '1.5': 6, '1.75': 7, '1/3': 8, '10.0': 9, '16.0': 10, '18.0': 11, '2.0': 12, '2.25': 13, '2.5': 14, '2.75': 15, '2/3': 16, '20.0': 17, '3.0': 18, '3.25': 19, '3.5': 20, '4.0': 21, '4.25': 22, '4.5': 23, '4/3': 24, '5.0': 25, '5.5': 

In [12]:
generate()

In Generate
In Prepare Sequences Prediction
In Generate Notes: netIn[start] = [67, 77, 39, 75]
['B-3', 'A4', 'G5', 'G4', 'G4', 'A4', 'G4', 'G#3', 'A4', 'A3', 'G5', 'A5', 'G4', 'F#4', '11.2', 'A4', 'G#3', 'A4', 'G4', 'G4', 'G5', 'G#3', 'D5', 'C3', 'G#5', 'A3', 'A4', 'G4', 'A4', 'A4', 'A4', 'A4', 'C3', 'D3', 'C3', 'G4', 'E-4', 'C3', 'G4', 'G4', 'G4', 'B-2', 'G4', 'A4', 'E-4', 'A4', 'A4', 'A2', 'G4', 'B-5', 'F3', 'A4', 'C#3', 'A4', 'G4', 'A4', 'G4', 'F#3', 'C#2', 'G4', 'A4', 'G#3', '8.1', 'B3', 'F3', 'A4', 'F4', 'A4', 'A4', 'B-3', 'D5', 'A4', 'C#2', 'C#2', 'C#3', 'F#3', 'F#3', 'A4', 'A4', 'F#3', 'A4', 'D3', 'A4', 'C3', 'F#3', 'D3', 'C3', 'G4', 'G4', 'G#4', 'G5', 'G5', 'C#5', 'G4', 'E-4', 'A4', 'F3', 'A4', 'C3', 'G4', 'B3', 'G4', 'G5', 'G#3', 'E-5', 'C3', 'A3', 'G4', 'A4', 'A4', 'G4', 'A4', 'A4', 'F3', 'G4', 'A2', '11.0', 'G4', 'B-2', 'F3', 'B-2', 'G4', 'G5', 'C3', 'G5', 'B-4', 'A4', 'A4', 'A4', 'A4', 'G4', 'A4', 'A4', 'F#3', 'C4', 'G5', 'C3', 'G4', 'A4', 'A3', '0', 'B-5', 'G#3', 'A4', 'A3

# Byron's code

In [ ]:
import glob
import numpy as np
import pandas as pd
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

def train_network():
    """ Train a Neural Network to generate music """
    # Get notes from midi files
    notes = get_notes()

    # Get the number of pitch names
    n_vocab = len(set(notes))

    # Convert notes into numerical input
    network_input, network_output = prepare_sequences(notes, n_vocab)

    # Set up the model
    model = create_network(network_input, n_vocab)
    history = History()
    
    # Fit the model
    n_epochs = 2
    model.summary()
    model.fit(network_input, network_output, callbacks=[history], epochs=n_epochs, batch_size=64)
    model.save('LSTMmodel.h5')
    
    # Use the model to generate a midi
    prediction_output = generate_notes(model, notes, network_input, len(set(notes)))
    create_midi(prediction_output, 'pokemon_midi')
    
    # Plot the model losses
    pd.DataFrame(history.history).plot()
    plt.savefig('LSTM_Loss_per_Epoch.png', transparent=True)
    plt.close()
    
def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in glob.glob("Music/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes
  
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)
  
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model
    
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(set(item for item in notes))
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
train_network()